In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 15.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp312-cp312-macosx_11_0_arm64.whl size=1155183 sha256=f25d06d960a328d132304910cf1ac0bbf33f843a06d1a44c0ea1e4c9cc5bb068
  Stored in directory: /Users/falonso11/Library/Caches/pip/wheels/b3/1c/42/0e26b8d512adc6bce10ff71a05229366b4ccec641cd3b42111
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.


In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.4 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
%sql sqlite:///my_data1.db

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [7]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [8]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [9]:
print(df.columns)

print(df['Launch_Site'].unique())

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')
['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']


In [10]:
print(df[df['Launch_Site'].str.startswith('CCA')].head(5))

         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

In [11]:
# 1. Create a filter (a boolean Series) for rows where Customer is 'NASA (CRS)'
#    We use == to check for an exact match
is_nasa_crs = df['Customer'] == 'NASA (CRS)'

# 2. Apply this filter to the DataFrame to get *only* the NASA (CRS) launches
nasa_crs_launches = df[is_nasa_crs]

# 3. From this new filtered DataFrame, select the 'PAYLOAD_MASS__KG_' column
#    (We know this column name from your first task's output)
payload_column = nasa_crs_launches['PAYLOAD_MASS__KG_']

# 4. Calculate the sum of all values in that column
total_payload_mass = payload_column.sum()

print("Total payload mass for NASA (CRS):", total_payload_mass)

Total payload mass for NASA (CRS): 45596


In [12]:
# Filter for the 'F9 v1.1' booster, select the payload column, and calculate its mean
average_payload = df[df['Booster_Version'] == 'F9 v1.1']['PAYLOAD_MASS__KG_'].mean()

print("Average payload mass for F9 v1.1:", average_payload)

Average payload mass for F9 v1.1: 2928.4


In [13]:
# Note: We are assuming the string for a successful ground pad landing is 
# 'Success (ground pad)'. To be 100% sure, you could run 
# print(df['Landing_Outcome'].unique()) first to see all possible values.

# Filter for successful ground pad landings, select the 'Date' column, and find the minimum (earliest) date
first_success_date = df[df['Landing_Outcome'] == 'Success (ground pad)']['Date'].min()

print("First successful ground pad landing date:", first_success_date)

First successful ground pad landing date: 2015-12-22


In [14]:
# 1. Create the first filter for the landing outcome
#    (We're assuming the string is 'Success (drone ship)')
f1_landing = df['Landing_Outcome'] == 'Success (drone ship)'

# 2. Create the second filter for payload mass > 4000
f2_payload_min = df['PAYLOAD_MASS__KG_'] > 4000

# 3. Create the third filter for payload mass < 6000
f3_payload_max = df['PAYLOAD_MASS__KG_'] < 6000

# 4. Combine all three filters using the '&' (AND) operator
#    Note: The parentheses around each filter are *required*!
combined_filter = (f1_landing) & (f2_payload_min) & (f3_payload_max)

# 5. Apply this single combined filter to the DataFrame,
#    then select the 'Booster_Version' column, and get the unique values
successful_boosters = df[combined_filter]['Booster_Version'].unique()

print(successful_boosters)

['F9 FT B1022' 'F9 FT B1026' 'F9 FT  B1021.2' 'F9 FT  B1031.2']


In [15]:
# 1. Select the 'Mission_Outcome' column
# 2. Call the .value_counts() method on it
mission_outcomes = df['Mission_Outcome'].value_counts()

# 3. Print the result
print(mission_outcomes)

Mission_Outcome
Success                             98
Failure (in flight)                  1
Success (payload status unclear)     1
Success                              1
Name: count, dtype: int64


In [16]:
# 1. The "subquery": Find the maximum payload mass.
#    The "suitable aggregate function" is .max()
max_payload = df['PAYLOAD_MASS__KG_'].max()

# 2. The "main query": Filter the DataFrame for all rows 
#    where the payload mass is *equal to* that maximum.
#    Then, select the 'Booster_Version' column and get the unique values.
boosters = df[df['PAYLOAD_MASS__KG_'] == max_payload]['Booster_Version'].unique()

print(boosters)

['F9 B5 B1048.4' 'F9 B5 B1049.4' 'F9 B5 B1051.3' 'F9 B5 B1056.4'
 'F9 B5 B1048.5' 'F9 B5 B1051.4' 'F9 B5 B1049.5' 'F9 B5 B1060.2 '
 'F9 B5 B1058.3 ' 'F9 B5 B1051.6' 'F9 B5 B1060.3' 'F9 B5 B1049.7 ']


In [17]:
# 1. Create the two filters required by the task
# Filter 1: The year is 2015 (translating the SQL hint)
filter_year = df['Date'].str.startswith('2015')

# Filter 2: The landing outcome is 'Failure (drone ship)'
# (As always, you might run df['Landing_Outcome'].unique() to confirm the exact string)
filter_landing = df['Landing_Outcome'] == 'Failure (drone ship)'

# 2. Combine the filters using the '&' (AND) operator
combined_filter = (filter_year) & (filter_landing)

# 3. Select the rows AND the columns we need using .loc
#    We select 'Date' so we can extract the month from it
columns_to_select = ['Date', 'Landing_Outcome', 'Booster_Version', 'Launch_Site']
filtered_df = df.loc[combined_filter, columns_to_select]

# 4. Create the new 'month' column
#    We make a .copy() to avoid a common (but harmless) SettingWithCopyWarning
final_df = filtered_df.copy()

# This is the Pandas version of SQL's substr(Date, 6, 2)
# It slices the string (e.g., '2015-06-22') at index 5 up to 7
final_df['month'] = final_df['Date'].str[5:7]

# 5. Display the final list of columns requested
output_columns = ['month', 'Landing_Outcome', 'Booster_Version', 'Launch_Site']
print(final_df[output_columns])

   month       Landing_Outcome Booster_Version  Launch_Site
13    01  Failure (drone ship)   F9 v1.1 B1012  CCAFS LC-40
16    04  Failure (drone ship)   F9 v1.1 B1015  CCAFS LC-40


In [18]:
# --- Best Practice: Do this first! ---
# 1. Ensure the 'Date' column is inthe proper datetime format.
#    This allows pandas to compare dates correctly.
#    (If you already did this in Task 9, you can skip this line)
df['Date'] = pd.to_datetime(df['Date'])

# --- Now, the one-line solution ---

# 2. Filter the DataFrame for the date range,
#    then select the 'Landing_Outcome' column,
#    and get the value counts.
outcome_counts = df[
    df['Date'].between('2010-06-04', '2017-03-20')
]['Landing_Outcome'].value_counts()

# 3. Print the results
print(outcome_counts)

Landing_Outcome
No attempt                10
Failure (drone ship)       5
Success (drone ship)       5
Controlled (ocean)         3
Success (ground pad)       3
Failure (parachute)        2
Uncontrolled (ocean)       2
Precluded (drone ship)     1
Name: count, dtype: int64
